

## 기본 패키지 및 데이터 정보 로드


In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")


import matplotlib.pyplot as plt
import missingno # 널값 바차트로 시각화

In [ ]:
netflix = pd.read_csv('./use_df.csv')

In [ ]:
netflix.head(1)

In [ ]:
netflix.columns

In [ ]:
df_net = netflix.drop(columns = ['Unnamed: 0'],axis =1)

In [ ]:
df_net.isnull().sum()

In [ ]:
color= ['dimgrey','dimgrey','blue','blue','blue','dimgrey','blue','blue','blue','blue','blue',
        'dimgrey','blue','blue','blue','blue','blue','blue','blue','blue','blue','blue','dimgrey',
        'blue','blue','blue']
missingno.bar(df_net,fontsize=10,color=color,figsize=(10,5))
plt.title('COLUMN MISSING VALUES',fontsize=20)

## 인코딩

viewpoint 예측에 사용될 변수들을 일단은 타입, 생산 국가, 상영 국가와 장르, 태그, 출시 일, 런타임, 연령, imdb 투표수와 점수, 로튼토마토의 숨겨진 명작 점수들을 가지고 먼저 예측초기모델을 만들어보자

null값 처리

In [ ]:
df = df_net[['Series or Movie', 'COUNTRY', 'Country Availability', 'Hidden Gem Score',
             'Release Date', 'GENRE','Tags',
             'View Rating', 'Runtime','VALUE','IMDb Score', 'IMDb Votes']]

In [ ]:
df.isnull().sum()

In [ ]:
df['Release Date'] = df['Release Date'].fillna(0)  # 날짜 데이터는 null값을 0으로 채움

In [ ]:
df["Release Date"] = pd.to_datetime(df['Release Date'])
df['month'] = df['Release Date'].dt.month
df = df.drop(columns = ['Release Date'],axis =1)

In [ ]:
df['Series or Movie'] = df['Series or Movie'].replace({'Movie': 1, 'TV': 2})  # type은 1,2로 인코딩

In [ ]:
df['Series or Movie'].value_counts() # 영화 데이터가 4배정도 많은 걸로 나타남

In [ ]:
df['COUNTRY'] = df['COUNTRY'].fillna('NA')  # 국가의 널값은 NA로 대체

In [ ]:
df['GENRE'] = df['GENRE'].fillna('NA') # 장르의 널값도 NA로 대체

In [ ]:
rows = len(df['Tags'])  # 태그는 태그 수를 사용
df['Tag_count'] = 0
for i in range(rows):
    df.Tag_count[i] = len(str(df.Tags.iloc[i]).split(','))
df = df.drop(['Tags'], axis = 1)

In [ ]:
rows = len(df['Country Availability'])  # 상영국가는 상영국가 수를 사용
df['C_count'] = 0
for i in range(rows):
    df.C_count[i] = len(str(df['Country Availability'].iloc[i]).split(','))
df = df.drop(['Country Availability'], axis = 1)

In [ ]:
for i in range(rows):
    if df['View Rating'][i] in ['PG', 'TV-PG', 'TV-G', 'TV-Y7', 'TV-Y', 'G', 'Unrated', 'TV-Y7-FV', 'Approved']:
        df['View Rating'][i] = 1  # 어린이 컨텐츠 = 1
    elif df['View Rating'][i] in ['PG-13', 'TV-14', 'R', 'GP', 'Passed', 'X']:
        df['View Rating'][i] = 2  # 청소년 컨텐츠 = 2
    elif df['View Rating'][i] in ['TV-MA', 'Not Rated','NC-17']:
        df['View Rating'][i] = 3  # 성인컨텐츠 = 3
    else:
        df['View Rating'][i] = 0 # null값을 0으로 처리


In [ ]:
df['View Rating'] = df['View Rating'].astype(int)

In [ ]:
df['Runtime'] = df['Runtime'].replace({'< 30 minutes' : 0,
                                             '30-60 mins' : 1,
                                             '1-2 hour' : 2,
                                             '> 2 hrs' : 3 })

In [ ]:
df['Runtime'] = df['Runtime'].astype(int)

In [ ]:
df['Runtime'].value_counts()

In [ ]:
df.isnull().sum()

In [ ]:
df = df.dropna(how='any',axis=0)  # hidden gem score, imdb score, imdb vote의 null값이 같으므로 널값이 있는 행을 지움

In [ ]:
df.info()

### 장르와 제작국가는 원핫인코딩으로 인코딩

In [ ]:
pd_df = pd.get_dummies(df[['COUNTRY','GENRE']])
df_result = pd.concat([df, pd_df], axis=1)

In [ ]:
df_result = df_result.drop(['COUNTRY','GENRE'], axis = 1)

In [ ]:
df_taget = df_result['VALUE']
df_result = df_result.drop(['VALUE'], axis = 1)

In [ ]:
df_result.head(1)

In [ ]:
df_taget.head(1)

## 딥러닝을 통해 모든 변수들을 넣고 예측

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
X = df_result.iloc[:].values
y = df_taget.iloc[:].values

In [ ]:
y = y.reshape(-1,1)

In [ ]:
y

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X)
y_train = sc.fit_transform(y)

In [ ]:
from keras import models
from keras import layers

In [ ]:
from keras.layers.core import Dropout
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(150, activation='relu',
                           input_shape=(X_train.shape[1],)))
    model.add(layers.Dense(280, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(99, activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(24, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

In [ ]:
X_train.shape

In [ ]:
import numpy as np

k = 4
all_mae_histories = []

num_val_samples = len(X_train) // k
num_epochs = 300
all_scores = []
for i in range(k):
    print('처리중인 폴드 #', i)
    val_data = X_train[i * num_val_samples: (i + 1) * num_val_samples]  # 검증 데이터 준비: k번째 분할
    val_targets = y_train[i * num_val_samples: (i + 1) * num_val_samples]

    partial_train_data = np.concatenate(  # 훈련 데이터 준비: 다른 분할 전체
        [X_train[:i * num_val_samples],
         X_train[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [y_train[:i * num_val_samples],
         y_train[(i + 1) * num_val_samples:]],
        axis=0)

model = build_model()  # 케라스 모델 구성(컴파일 포함)
history = model.fit(partial_train_data, partial_train_targets,  # 모델 훈련(verbose=0이므로 훈련 과정이 출력되지 않습니다.)
                    validation_data=(val_data, val_targets),
                    epochs=num_epochs, batch_size=1, verbose=True)
mae_history = history.history['val_mae']
all_mae_histories.append(mae_history)

val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)  # 검증 세트로 모델 평가
all_scores.append(val_mae)

In [ ]:
all_scores

In [ ]:
average_mae_history = [
    np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(1, len(average_mae_history) + 1), average_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()

In [ ]:
def smooth_curve(points, factor=0.9):
    smoothed_points = []
    for point in points:
        if smoothed_points:
            previous = smoothed_points[-1]
            smoothed_points.append(previous * factor + point * (1 - factor))
        else:
            smoothed_points.append(point)
    return smoothed_points

smooth_mae_history = smooth_curve(average_mae_history[10:])

plt.plot(range(1, len(smooth_mae_history) + 1), smooth_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()

**에포크 300으로 딥러닝을 진행했을 때 오차가 0.227, 227정도로 viewpoint가 1~40000인것을 감안하면 오차가 좀 큰걸로 나옴**
- all_scores = [0.2270810753107071]

In [ ]:
num_epochs = 150
all_mae_histories = []
for i in range(k):
    print('처리중인 폴드 #', i)
    val_data = X_train[i * num_val_samples: (i + 1) * num_val_samples]  #검증 데이터 준비: k번째 분할    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
    partial_train_data = np.concatenate(  # 훈련 데이터 준비: 다른 분할 전체
        [X_train[:i * num_val_samples],
         X_train[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [y_train[:i * num_val_samples],
         y_train[(i + 1) * num_val_samples:]],
        axis=0)

model = build_model()  # 케라스 모델 구성(컴파일 포함)
history = model.fit(partial_train_data, partial_train_targets,  # 모델 훈련(verbose=0이므로 훈련 과정이 출력되지 않습니다.)
                    validation_data=(val_data, val_targets),
                    epochs=num_epochs, batch_size=1, verbose=True)
mae_history = history.history['val_mae']
all_mae_histories.append(mae_history)

In [ ]:
mae_history = history.history['val_mae']
all_mae_histories.append(mae_history)

In [ ]:
average_mae_history = [
    np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(1, len(average_mae_history) + 1), average_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()

In [ ]:
def smooth_curve(points, factor=0.9):
    smoothed_points = []
    for point in points:
        if smoothed_points:
            previous = smoothed_points[-1]
            smoothed_points.append(previous * factor + point * (1 - factor))
        else:
            smoothed_points.append(point)
    return smoothed_points

smooth_mae_history = smooth_curve(average_mae_history[10:])

plt.plot(range(1, len(smooth_mae_history) + 1), smooth_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()

에포크 90정도 지점부터 오차가 줄어들지 않는것으로 보인다. 이 지점이 과대적합 지점이라고 생각하고 새로운 모델을 만들어서 테스트 해보았다.

In [ ]:
model = build_model()  # 새롭게 컴파일된 모델을 얻습니다.
model.fit(X_train, y_train,  # 전체 데이터로 훈련시킵니다.
          epochs=90, batch_size=16, verbose=0)
test_mse_score, test_mae_score = model.evaluate(X_train, y_train)

In [ ]:
# batch_size = 1
# test_mse_score, test_mae_score = model.evaluate(X_train, y_train)
# 89/89 [==============================] - 0s 2ms/step - loss: 1.0165 - mae: 0.3206
# test_mae_score = 0.3206084668636322

In [ ]:
test_mae_score

##### 최종 결과
- batch_size = 16 , epochs = 90

- 89/89 [==============================] - 0s 2ms/step - loss: 0.2012 - mae: 0.1685
- test_mae_score = 0.1685454249382019

- 최종 결과 epochs 90지점에서 오차가 0.169정도로 나왔고, 최종 오차는 169정도 나는걸로 보인다.

- 데이터셋이 부족하여 오차를 더 줄이기는 어려워 보인다.
- 추후에 데이터셋을 더 추가하고, 딥러닝 코드를 손을 보면 더 좋은 예측모델이 나올것으로 보인다.

In [ ]:
# pd_df = pd.get_dummies(df_net['Tags'])
# df_result = pd.concat([df_net, pd_df], axis=1)